### STA 141B: Homework 4
Winter 2018

## Student Information

After the colons (in the same line) please write just your first name, last name, and the 9 digit student ID number below.

First Name: Timothy

Last Name: Murphy

Student ID: 912614348

## Instructions

We use a script that extracts your answers by looking for cells in between the cells containing the exercise statements.  So you 

- MUST add cells in between the exercise statements and add answers within them and
- MUST NOT modify the existing cells, particularly not the problem statement

To make markdown, please switch the cell type to markdown (from code) - you can hit 'm' when you are in command mode - and use the markdown language.  For a brief tutorial see: https://daringfireball.net/projects/markdown/syntax


## Requesting freshness

In this portion, you'll combine the assignment 3 data set with nutrition data from the [USDA Food Composition Databases](https://ndb.nal.usda.gov/ndb/search/list). The CSV file `fresh.csv` contains the fresh fruits and vegetables data you extracted in assignment 3.

The USDA Food Composition Databases have a [documented](https://ndb.nal.usda.gov/ndb/doc/index) web API that returns data in JSON format . You need a key in order to use the API. Only 1000 requests are allowed per hour, so it would be a good idea to use [caching][requests_cache].

[Sign up for an API key here](https://api.data.gov/signup/). The key will work with any Data.gov API. You may need the key again later in the quarter, so make sure you save it.

These modules may be useful:

* [requests](http://docs.python-requests.org/en/master/user/quickstart/)
* [requests_cache][]
* [urlparse](https://docs.python.org/2/library/urlparse.html)
* [pandas](http://pandas.pydata.org/pandas-docs/stable/)

[requests_cache]: https://pypi.python.org/pypi/requests-cache

__Exercise 1.1__ (20 pts)

Read the [search request documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-SEARCH.md), then write a function called `ndb_search()` that makes a search request. The function should accept the search term as an argument. The function should return the search result items as a list (for 0 items, return an empty list).

Note that the search url is: `https://api.nal.usda.gov/ndb/search`

As an example, a search for `"quail eggs"` should return this list:

```python
[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  u'ndbno': u'45094707',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'L&W, QUAIL EGGS, UPC: 024072000256',
  u'ndbno': u'45094890',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  u'ndbno': u'45099560',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  u'ndbno': u'45169279',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u"D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  u'ndbno': u'45178254',
  u'offset': 4},
 {u'ds': u'SR',
  u'group': u'Dairy and Egg Products',
  u'name': u'Egg, quail, whole, fresh, raw',
  u'ndbno': u'01140',
  u'offset': 5}]
```

As usual, make sure you document and test your function.

In [1]:
import os
import pandas as pd
import requests
import requests_cache
import urllib3
import itertools
import plotnine as gg
from sklearn.preprocessing import StandardScaler

/Users/tmm/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def ndb_search(key, search_term):
    """
    Purpose:
    Makes a search request from the USDA Food Composition API
    
    Args:
    key : Data.gov API key , search_term : search term for USDA Food Composition API
    
    Returns:
    A list based on the search_term input (for 0 items, return an empty list)
    """
    parameters = { "api_key" : key, "q" : search_term, "format" : "json"} # set the parameters for API
    url = "https://api.nal.usda.gov/ndb/search" # set the API url
    req = requests.get(url, params = parameters) # make the request
    js = req.json() # convert to json
    
    # check for errors
    if "error" in js:
        req = []
    
    # check for 0's
    elif "list" in js:
        req = js["list"]["item"]
        if len(req) == 0:
            req = []
    
    # return empyt list
    else:
        req = [] # return empy list
        
    return req # return the request

gov_key = "gYdxeG8pCE56WkormQ8vxOpzwzX5sBEFmrVfiiUI"
js_search = ndb_search(gov_key, "quail eggs")
js_search

[{'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  'ndbno': '45169279',
  'offset': 0},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': "D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  'ndbno': '45178254',
  'offset': 1},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  'ndbno': '45094707',
  'offset': 2},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'L&W, QUAIL EGGS, UPC: 024072000256',
  'ndbno': '45094890',
  'offset': 3},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  'ndbno': '45099560',
  'offset': 4},
 {'ds': 'SR',
  'group': 'Dairy and Egg Products',
  'name': 'Egg, quail, whole, fresh, raw',
  'ndbno': '01140',
  'offset': 5}]

__Exercise 1.2__ (20 pts)

Use your search function to get NDB numbers for the foods in the `fresh.csv` file. It's okay if you don't get an NDB number for every food, but try to come up with a strategy that gets most of them. Discuss your strategy in a short paragraph.

Hints:

* The foods are all raw and unbranded.
* You can test search terms with the [online search page](https://ndb.nal.usda.gov/ndb/search/list).
* You can convert the output of `ndb_search()` to a data frame with `pd.DataFrame()`.
* The string methods for [Python](https://docs.python.org/2/library/stdtypes.html#string-methods) and [Pandas](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary) are useful here. It's okay if you use _simple_ regular expressions in the Pandas methods, although this exercise can be solved without them.
* You can merge data frames that have a column in common with `pd.merge()`.

## The Strategy ##
My primary strategy for retrieving the NDB numbers for the foods in the fresh.csv file is to first create a function that will be easily callable for future use. I then looped through each produce item in the fresh dataframe. The food item values that are two words in the dataframe have an underscore seperating them which is not valid when using the NDB API. So using the .replace() function I swapped the underscore with a space in order to capture the two-word produce items. Next, using the ndb_search function along with the inclusion of a raw search term given your specifications. I was able to narrow the search items requested and exclude non-raw produce items within the NDB database. In order to avoid unnecessary errors, I used try/except statements to account for items such as "kiwi" and "tangerines" that are incorrectly spelled in the fresh.csv file. I then refined the resulting search dataframe further by filtering out branded produce items. 

This was a good strategy with the only issue being it's handling of identical food items with a different form. The identical foods with different forms resulted in the peeled/unpeeled cucumber and green/red cabbage having the same NDB number. 

In [3]:
def ndbno_search(fresh, ndb, key):
    """
    Purpose:
    Using the search function get the NDB number for the foods in the fresh.csv file
    
    Args:
    fresh : fruit/vegetable dataframe, ndb : NDB dataframe, key : key for API
    
    Returns:
    A dataframe consisting of the food items and their corresponding NDB number
    """
    
    error_items = {"kiwi" : "kiwifruit", "tangerines" : "tangerine mandarins"} # hard code
    
    # loop through the fresh dataframe and obtain the NDB number through API requests
    for item in fresh["food"]:
        item_swap = item.replace("_", " ")
        item_swap = error_items.get(item_swap, item_swap)
        item_df = pd.DataFrame(ndb_search(key, item_swap + ", raw"))
       
        # filter the API requests to the best choice for our item
        try:
            item_df = item_df[item_df["ds"] == "SR"]
            item_df = item_df.iloc[:1,:]
            ndb = ndb.append({"food" : item, "ndbno" : item_df.iloc[0,3]}, ignore_index = True)
            
        # return NULL  for ndbno if exception
        except:
            ndb = ndb.append({"food" : item, "ndbno" : "NULL"}, ignore_index = True)
    
    return ndb

fresh_df = pd.read_csv("fresh.csv") 
empty_ndb = pd.DataFrame()
ndb = ndbno_search(fresh_df, empty_ndb, gov_key)
ndb = ndb.drop_duplicates()
cfresh_df1 = fresh_df.join(ndb.set_index("food"), on ="food") # join the NDB and Fresh dataframes

# reorder columns
cfresh_df1 = cfresh_df1[["type", "food", "ndbno", "form", "price_per_lb", "yield", "lb_per_cup", "price_per_cup"]]
cfresh_df1

,type,food,ndbno,form,price_per_lb,yield,lb_per_cup,price_per_cup
0,fruit,watermelon,09326,Fresh1,0.333412,0.520000,0.330693,0.212033
1,fruit,cantaloupe,09181,Fresh1,0.535874,0.510000,0.374786,0.393800
2,fruit,tangerines,09218,Fresh1,1.377962,0.740000,0.407855,0.759471
3,fruit,strawberries,09316,Fresh1,2.358808,0.940000,0.319670,0.802171
4,fruit,plums,09279,Fresh1,1.827416,0.940000,0.363763,0.707176
5,fruit,oranges,09201,Fresh1,1.035173,0.730000,0.407855,0.578357
6,fruit,raspberries,09302,Fresh1,6.975811,0.960000,0.319670,2.322874
7,fruit,pomegranate,09286,Fresh1,2.173590,0.560000,0.341717,1.326342
8,fruit,pineapple,09266,Fresh1,0.627662,0.510000,0.363763,0.447686
9,fruit,apricots,09021,Fresh1,3.040072,0.930000,0.363763,1.189102


__Exercise 1.3__ (20 pts)

Read the [food reports V2 documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md), then write a function called `ndb_report()` that requests a _basic_ food report. The function should accept the NDB number as an argument and return the list of nutrients for the food.

Note that the report url is: `https://api.nal.usda.gov/ndb/V2/reports`

For example, for `"09279"` (raw plums) the first element of the returned list should be:

```python
{u'group': u'Proximates',
 u'measures': [{u'eqv': 165.0,
   u'eunit': u'g',
   u'label': u'cup, sliced',
   u'qty': 1.0,
   u'value': u'143.93'},
  {u'eqv': 66.0,
   u'eunit': u'g',
   u'label': u'fruit (2-1/8" dia)',
   u'qty': 1.0,
   u'value': u'57.57'},
  {u'eqv': 151.0,
   u'eunit': u'g',
   u'label': u'NLEA serving',
   u'qty': 1.0,
   u'value': u'131.72'}],
 u'name': u'Water',
 u'nutrient_id': u'255',
 u'unit': u'g',
 u'value': u'87.23'}
```

Be sure to document and test your function.

In [ ]:
def ndb_report(ndbno):
    """
    Purpose:
    Makes a basic food report request from the USDA Food Composition API 
    
    Args:
    ndbno : NDB number 
    
    Returns:
    List of nutrients for the food
    """
    global gov_key # bring in the API key globally because question says can all have one argument
    parameters = { "api_key" : gov_key, "ndbno" : ndbno, "type" : "b", "format" : "json"} # define parameters
    url = "https://api.nal.usda.gov/ndb/V2/reports" # API url
    req = requests.get(url, params = parameters) # make API request
    js = req.json()["foods"][0]["food"]["nutrients"] # pull relevant data and store in json format
    
    return js



In [ ]:
def nutrients(fresh):
    """
    Purpose:
    Builds a nutrient dataframe using the <nfresh_df> and <ndb_report> data
    
    Args:
    fresh : dataframe containing produce data and NDB numbers
    
    Returns:
    nuts : dataframe containing nutrient information corresponding to each produce item
    
    """
    nex = pd.DataFrame()
    nuts = pd.DataFrame()
    
    # iterate through the fresh rows using the ndbno number and obtain the nutrional data
    for _ , fresh_row in fresh.iterrows():
        
        ndb_number = fresh_row["ndbno"]
        item = ndb_report(ndb_number)
        nex["food"] = [fresh_row["food"]]

        for key in item:
            nex[key["name"]] = [key["value"]]

        nuts = nuts.append(nex)
    nuts = nuts.reset_index(drop = True)
    
    return nuts

nutrient_df = nutrients(cfresh_df1)

# rename the nutritional column headers to join on daily value data later
nutrient_df = nutrient_df.rename({"Total lipid (fat)" : "Total_Fat",
                                  "Carbohydrate, by difference" : "Total_carbohydrate",
                                  "Fiber, total dietary" : "Fiber",
                                  "Calcium, Ca" : "Calcium",
                                  "Iron, Fe" : "Iron",
                                  "Magnesium, Mg" : "Magnesium",
                                  "Phosphorus, P" : "Phosphorus",
                                  "Potassium, K" : "Potassium",
                                  "Sodium, Na" : "Sodium",
                                  "Zinc, Zn" : "Zinc",
                                  "Vitamin C, total ascorbic acid" : "Vitamin_C",
                                  "Vitamin B-6" : "Vitamin_B6",
                                  "Folate, DFE" : "Folate",
                                  "Vitamin B-12" : "Vitamin_B12",
                                  "Vitamin A, IU" : "Vitamin_A",
                                  "Vitamin E (alpha-tocopherol)" : "Vitamin_E",
                                  "Vitamin K (phylloquinone)" : "Vitamin_K",
                                  "Fatty acids, total saturated" : "Saturated_fatty_acids"},
                                axis = "columns")
pd.set_option('display.max_columns', None)
nutrient_df

__Exercise 1.4__ (20 pts) Which foods provide the best combination of price, yield, and nutrition? You can use kilocalories as a measure of "nutrition" here, but more a detailed analysis is better. Use plots to support your analysis.

In [ ]:
# hard code in the daily value nutritional data
nutrient_names = ["Total_Fat" ,"Saturated_fatty_acids","Cholesterol","Sodium","Potassium","Total_carbohydrate",
                  "Fiber","Protein","Vitamin_A","Vitamin_C","Calcium","Iron","Vitamin_D","Vitamin_E","Vitamin_K",
                  "Thiamin","Riboflavin","Niacin","Vitamin_B6","Folate","Vitamin_B12","Biotin","Pantothenic_acid",
                  "Phosphorus","Iodine","Magnesium","Zinc","Selenium","Copper","Manganese","Chromium","Molybdenum",
                  "Chloride"]

nutrient_dv = [65,20,300,2400,3500,300,25,50,5000,60,1000,18,400,30,80,1.5,1.7,20,
               2,400,6,300,10,1000,150,400,15,70,2,2,120,75,3400]

# create dictionary containing the nutrient key name and its corresponding daily value based on 2000 calories
nutrient_dict = dict(zip(nutrient_names, nutrient_dv))

In [ ]:
nutrient_df1 = nutrient_df.copy()
nutrient_df1_headers = list(nutrient_df1)
daily_value_headers = nutrient_names

# determine which nutrients in our API requests can be kept for daily value analysis 
keep = [ name for name in nutrient_df1_headers if name in daily_value_headers ]

# create new nutrient dataframe with relevant nutrients and calorie amount
nutrient_df1 = nutrient_df1[["food"] + ["Energy"] + keep]

In [ ]:
nutrient_df2 = nutrient_df1.copy()

# convert nutrient data frame to numeric for calculations 
nutrient_df2 = nutrient_df2.apply(pd.to_numeric, errors = "ignore")
headers = list(nutrient_df2.iloc[:,1:])

In [ ]:
score_df = pd.DataFrame() # initialize a dataframe which will hold the nutritional scores temporarily 

# iterate through the nutritional dataframe skipping the first column 
for index, row in nutrient_df2.iloc[:,1:].iterrows():
    # initialize single row score dataframe which will be appended later
    next_score = pd.DataFrame()
    
    # loop through the headers and find key value in the nutrient dictionary and then perform calculations
    for name in headers:
        try:
            nut_value = nutrient_dict.get(name) # get the value corresponding the key value in the nutrient dict
            next_score[name] = [row[name] / nut_value]  # append to the next score dataframe
        
        except:
            next_score[name] = [row["Energy"]] # account for the energy column
            
    # calculate the final nutrional score        
    next_score["Score"] = (next_score.iloc[:,1:].sum(axis = 1)) / next_score["Energy"] * 100
    
    score_df = score_df.append(next_score) # append to the nutrional score dataframe

# reorder the columns dataframe
score_df["food"] = list(nutrient_df2["food"])
score_headers = list(score_df)
score_headers = score_headers[-1:] + score_headers[:-1]
score_df = score_df[score_headers]
score_df = score_df[["food", "Energy", "Score"]]

In [ ]:
final_headers = fresh_df[["food", "type", "price_per_lb", "yield"]]
final_df = pd.merge(score_df, final_headers, on = "food")
final_df = final_df[["food", "Energy", "type", "price_per_lb", "yield", "Score"]] # create final dataframe
final_df["Value"] = ((1 / final_df["price_per_lb"]) / final_df["yield"]) # calculate the value score
final_df.columns = map(str.lower, final_df.columns) # lower case the headers for convenience 

In [ ]:
# normalize the nutrional scores and value scores
scaler_obj = StandardScaler()
score_transform = scaler_obj.fit_transform(final_df[["score", "value"]])
final_df["nscore"] = score_transform[:,0]
final_df["nvalue"] = score_transform[:,1]
final_df = final_df.drop_duplicates() # drop any duplicates 

## Data Filtering, Scoring, and Strategy
My strategy for conducting this analysis was to first explore the data in order to determine the optimal method for making the optimal choice for best combination of price, yield, and nutrition. My intitial findings conclude that there are some issues within the given "fresh.csv" for the names of the produce items. For example, "kiwi" should be named "kiwifruit" and "tangerines" should be named "tangerine mandarins" in order to pull the correct NDB number and nutritional information. I accounted for this mishap through hard coding the correct names into my ndbno_search() function. After collecting all the nutrional data to the best of my ability I then merged this with the given fresh file to create an aggregate picture of the retail and nutrional value for each item. 

From here it was tough figuring out how to compare different items based on nutrional value of different amounts so I decided that the best way to do this was to bring in the FDA daily value recommendations for each nutrient. I then divided each produce items nutrient value by the FDA recommended daily value amount to get a percentage of nutrient needs met based on a 100 gramm equivalent. Then I divided by the kcal amount to obtain a per calorie basis and multiplied by 100 for scaling. Finally, I summed across all nutrient scores to obtain a single value for each produce item which gave a total nutritional score for each item. From here I normalized the nutrional score across all items to obtain its number of standard deviations away from the mean (higher - better nutrition, lower - worse nutrition). Next I used the same calculation conducted in homework 3 ( 1 / price_per_lb / yield) in order to obtain a price score across all items. This allowed for a single score accounting for price and yield that made it possible to compare each item against eachother. As I did with the nutrional score, I also normalize the value score as well to obtain the number of standard deviations away from the mean value score. 

Now that I had the original fresh.csv data and normalized value and nutrional scores I can begin visualizing the data on a two dimensional plane. 

In [ ]:
final_df

In [ ]:
Price_Vs_Nutrition = (gg.ggplot(final_df, gg.aes(x = "nscore", y = "price_per_lb", color = "type", size = 4)) 
                      + gg.geom_point(alpha = 0.5, show_legend = {"size" : False}) 
                      + gg.geom_text(gg.aes(label = "food"), color = "black", size = 6)
                      + gg.theme(figure_size = (10,10))
                      + gg.labs(x = "Normalized Nutrition Score", 
                                y = "Price Per Lb", 
                                title = "Price Per Lb vs. Normalized Nutrition Score"))

## Price Per Pound Against Normalized Nutrition Score

Using a scatter plot I am able to compare the normalized nutrional score and price per pound and color each point by the produce type ("fruit", "vegetables"). 

In [ ]:
Price_Vs_Nutrition

In [ ]:
pvn = final_df.copy()
pvn.set_index("food", inplace=True)
pvn = pvn.loc[["kale", "collard_greens", "turnip_greens", "mustard_greens", "cabbage"]] ; pvn

### Findings 

In doing so I found that the items with the highest nutritional value and for the lowest price were not surprising, they were a group of six listed above. These items stood out from all the rest due to their high nutrional value score which was above two standard deviations away from the mean and were relatively cheap. For example, "Kale" is packs a nutritious punch and is extremely cheap making it your best bet in terms of value and nutritious gains. 

Here the fruit types are usually price very high and don't hold much nutritional value when compared to their vegetable counterparts. This isn't to say they aren't healthy but when doing a comparison such as this they do not hold their own and are expensive. 

Note: the reason for their being duplicate cabbages is that there are a green and red cabbage but it is listed as just cabbage in the Fresh.csv file. 

In [ ]:
Yield_Vs_Nutrition = (gg.ggplot(final_df, gg.aes(x = "nscore", y = "yield", color = "type", size = 4)) 
                      + gg.geom_point(alpha = 0.5, show_legend = {"size" : None}) 
                      + gg.geom_text(gg.aes(label = "food"), color = "black", size = 6)
                      + gg.theme(figure_size = (10,10))
                      + gg.labs(x = "Normalized Nutrition Score", 
                                y = "Yield", 
                                title = "Yield vs. Normalized Nutrition Score"))

## Yield Against Normalized Nutrition Score

Using a scatter plot I am able to compare the normalized nutrional score and yield and color each point by the produce type ("fruit", "vegetables"). 

In [ ]:
Yield_Vs_Nutrition

In [ ]:
yvn = final_df.copy()
yvn.set_index("food", inplace=True)
yvn = yvn.loc[["kale","collard_greens", "turnip_greens", "mustard_greens", "cabbage"]] ; yvn

### Findings 

In doing so I found that the items with the highest nutritional value and for the best yield were not surprising, they were a group of six listed above. These items stood out from all the rest due to their high nutrional value score which was above two standard deviations away from the mean and had an extremely high yeild. 

Here, the fruits once again hold little nutritional value compared to the vegetables but have an extremely high yield. But when it comes to both the vegetables beat the fruit when it comes to a combination of both. 

Something to make note of here was that the file Fresh.csv contained yeilds above 100% which in real life does not make a lot of sense so that can attribute the very high yield scores. But none the less, I imagine this vegetable products have extremely high yields anyway. 